# CNNs

Team: HáLab

Team members: Stumphauser Nóra, Lestyan Bence

Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#load the images and labels
images = np.load('images.npy')
label = pd.read_csv('label.csv')
label = label.drop(label.columns[0], axis = 1)

In [3]:
#creating the train, valid and test data
x_train, x_test, y_train, y_test = train_test_split(images, label['label'], test_size=0.30, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=42)

In [4]:
print(len(x_train), len(x_valid), len(x_test))

2802 601 601


In [5]:
print(len(y_train), len(y_valid), len(y_test))

2802 601 601


In [6]:
x_train.shape

(2802, 7, 24)

In [7]:
y_train.shape

(2802,)

In [8]:
#reshaping the images, because input shape of KERAS is 3-dimentional, so we need the "channel dimension" (it is 1)
x_train = x_train.reshape(x_train.shape + (1,))
x_valid = x_valid.reshape(x_valid.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# The models

In [9]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import AUC
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf 

Using TensorFlow backend.


In [10]:
#defining early stopping, model checkpoint to save best models and reduce LR
earlyStopping = EarlyStopping(monitor='val_AUC', patience=10, verbose=0, mode='max')
mcp_save = ModelCheckpoint('models/BEST.mdl_wts.hdf5', save_best_only=True, monitor='val_AUC', mode='max')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_AUC', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='max')

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


#### First model - basic:

In [11]:
#we constructed a "random" model (random = not a very special)
model = Sequential()
model.add(Conv2D(32, kernel_size =(3, 3), strides =(1, 1), activation ='relu',input_shape=(7, 24, 1))) 
#model.add(MaxPooling2D(pool_size =(2, 2))) 
model.add(Conv2D(64, (3, 3), activation ='relu'))
model.add(Conv2D(64, (3, 3), activation ='relu'))
#model.add(MaxPooling2D(pool_size =(2, 2))) 
model.add(Flatten()) 
model.add(Dense(128, activation ='relu')) 
model.add(Dense(1, activation ='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['AUC'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 5, 22, 32)         320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 20, 64)         18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               147584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 203,457
Trainable params: 203,457
Non-trainable params: 0
__________________________________________________

In [14]:
#we training the model with callbacks
model.fit(x_train, y_train, batch_size=120, epochs=40, validation_data=(x_valid,y_valid), verbose=2, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Train on 2802 samples, validate on 601 samples
Epoch 1/40
2802/2802 - 2s - loss: 3.9966 - AUC: 0.4880 - val_loss: 0.7239 - val_AUC: 0.4887
Epoch 2/40
2802/2802 - 1s - loss: 0.7099 - AUC: 0.5198 - val_loss: 0.6903 - val_AUC: 0.5477
Epoch 3/40
2802/2802 - 1s - loss: 0.6810 - AUC: 0.5846 - val_loss: 0.6897 - val_AUC: 0.5831
Epoch 4/40
2802/2802 - 1s - loss: 0.6664 - AUC: 0.6276 - val_loss: 0.6690 - val_AUC: 0.6207
Epoch 5/40
2802/2802 - 1s - loss: 0.6396 - AUC: 0.6791 - val_loss: 0.6640 - val_AUC: 0.6501
Epoch 6/40
2802/2802 - 1s - loss: 0.6265 - AUC: 0.6981 - val_loss: 0.6639 - val_AUC: 0.6513
Epoch 7/40
2802/2802 - 1s - loss: 0.6085 - AUC: 0.7223 - val_loss: 0.6413 - val_AUC: 0.6783
Epoch 8/40
2802/2802 - 1s - loss: 0.6011 - AUC: 0.7302 - val_loss: 0.6471 - val_AUC: 0.6727
Epoch 9/40
2802/2802 - 1s - loss: 0.5868 - AUC: 0.7489 - val_loss: 0.6703 - val_AUC: 0.6608
Epoch 10/40
2802/2802 - 1s - loss: 0.5773 - AUC: 0.7598 - val_loss: 0.6516 - val_AUC: 0.6731
Epoch 11/40
2802/2802 - 1s - los

In [15]:
# let see AUC on test data
print('AUC on test data - last step:',model.evaluate(x_test, y_test, verbose=0)[1])
model4 = tf.keras.models.load_model('models/BEST.mdl_wts.hdf5')
print('AUC on test data - best model:',model4.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.70935297
AUC on test data - best model: 0.7024112


In [16]:
# We can manually save the model, if it is goood enough

In [31]:
model.save("models/custom_train86_test71.h5")

#### Second model - DL-1:

In [17]:
# it is a modified DL-1 model from https://arxiv.org/ftp/arxiv/papers/1604/1604.05377.pdf
model2 = Sequential()
model2.add(Conv2D(128, kernel_size = (1, 5), strides = (1, 1), activation ='relu',input_shape=(7, 24, 1))) 
model2.add(Conv2D(64, kernel_size = (7, 1), strides = (1, 1), activation ='relu'))
model2.add(MaxPooling2D(pool_size =(1, 2))) 
model2.add(Flatten()) 
model2.add(Dense(256, activation ='relu')) 
model2.add(Dense(1, activation ='sigmoid'))

In [18]:
model2.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['AUC'])

In [19]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 7, 20, 128)        768       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 20, 64)         57408     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 10, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               164096    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 222,529
Trainable params: 222,529
Non-trainable params: 0
________________________________________________

In [20]:
model2.fit(x_train, y_train, batch_size=120, epochs=40, validation_data=(x_valid,y_valid), verbose=2, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Train on 2802 samples, validate on 601 samples
Epoch 1/40
2802/2802 - 2s - loss: 4.8636 - AUC: 0.4855 - val_loss: 0.7287 - val_AUC: 0.5425
Epoch 2/40
2802/2802 - 1s - loss: 0.6943 - AUC: 0.5461 - val_loss: 0.6929 - val_AUC: 0.5446
Epoch 3/40
2802/2802 - 1s - loss: 0.6783 - AUC: 0.5960 - val_loss: 0.6879 - val_AUC: 0.5792
Epoch 4/40
2802/2802 - 1s - loss: 0.6653 - AUC: 0.6349 - val_loss: 0.6833 - val_AUC: 0.5992
Epoch 5/40
2802/2802 - 1s - loss: 0.6561 - AUC: 0.6464 - val_loss: 0.6753 - val_AUC: 0.6144
Epoch 6/40
2802/2802 - 1s - loss: 0.6366 - AUC: 0.6847 - val_loss: 0.6690 - val_AUC: 0.6384
Epoch 7/40
2802/2802 - 1s - loss: 0.6153 - AUC: 0.7151 - val_loss: 0.6851 - val_AUC: 0.6328
Epoch 8/40
2802/2802 - 1s - loss: 0.5915 - AUC: 0.7485 - val_loss: 0.6863 - val_AUC: 0.6366
Epoch 9/40
2802/2802 - 1s - loss: 0.5680 - AUC: 0.7710 - val_loss: 0.6782 - val_AUC: 0.6544
Epoch 10/40
2802/2802 - 1s - loss: 0.5652 - AUC: 0.7739 - val_loss: 0.6823 - val_AUC: 0.6393
Epoch 11/40
2802/2802 - 1s - los

In [21]:
print('AUC on test data - last step:',model2.evaluate(x_test, y_test, verbose=0)[1])
model4 = tf.keras.models.load_model('models/BEST.mdl_wts.hdf5')
print('AUC on test data - best model:',model4.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.69707406
AUC on test data - best model: 0.7024112


model2.save("models/dl-1_train88_test71.h5")

#### Third model - DL-2:

In [22]:
# it is a modified DL-2 model from https://arxiv.org/ftp/arxiv/papers/1604/1604.05377.pdf
model3 = Sequential()
model3.add(Conv2D(128, kernel_size = (2, 10), strides = (1, 1), activation ='relu',input_shape=(7, 24, 1))) 
#model3.add(Dropout(0.25))
#model3.add(MaxPooling2D(pool_size =(1, 2))) 
model3.add(Conv2D(64, kernel_size = (6, 1), strides = (1, 1), activation ='relu'))
model3.add(MaxPooling2D(pool_size =(1, 2))) 
model3.add(Dropout(0.25))
model3.add(Flatten()) 
model3.add(Dense(128, activation ='relu')) 
model3.add(Dropout(0.2))
#model3.add(Dense(64, activation ='relu')) 
#model3.add(Dropout(0.2))
model3.add(Dense(32, activation ='relu')) 
#model3.add(Dropout(0.2)) #-
model3.add(Dense(1, activation ='sigmoid'))

In [23]:
model3.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['AUC'])

In [24]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 6, 15, 128)        2688      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 15, 64)         49216     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 7, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               57472     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [25]:
model3.fit(x_train, y_train, batch_size=120, epochs=150, validation_data=(x_valid,y_valid), verbose=2, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Train on 2802 samples, validate on 601 samples
Epoch 1/150
2802/2802 - 2s - loss: 1.8957 - AUC: 0.5116 - val_loss: 0.7150 - val_AUC: 0.4767
Epoch 2/150
2802/2802 - 0s - loss: 0.7076 - AUC: 0.5047 - val_loss: 0.6922 - val_AUC: 0.4988
Epoch 3/150
2802/2802 - 0s - loss: 0.6926 - AUC: 0.5115 - val_loss: 0.6914 - val_AUC: 0.5248
Epoch 4/150
2802/2802 - 0s - loss: 0.6913 - AUC: 0.5388 - val_loss: 0.6903 - val_AUC: 0.5306
Epoch 5/150
2802/2802 - 0s - loss: 0.6894 - AUC: 0.5403 - val_loss: 0.6918 - val_AUC: 0.5032
Epoch 6/150
2802/2802 - 0s - loss: 0.6836 - AUC: 0.5524 - val_loss: 0.6893 - val_AUC: 0.5542
Epoch 7/150
2802/2802 - 0s - loss: 0.6849 - AUC: 0.5848 - val_loss: 0.6852 - val_AUC: 0.5826
Epoch 8/150
2802/2802 - 0s - loss: 0.6900 - AUC: 0.5502 - val_loss: 0.6903 - val_AUC: 0.5347
Epoch 9/150
2802/2802 - 0s - loss: 0.6787 - AUC: 0.5774 - val_loss: 0.6934 - val_AUC: 0.5518
Epoch 10/150
2802/2802 - 0s - loss: 0.6822 - AUC: 0.5766 - val_loss: 0.6912 - val_AUC: 0.5361
Epoch 11/150
2802/2802

In [26]:
print('AUC on test data - last step:',model3.evaluate(x_test, y_test, verbose=0)[1])
model4 = tf.keras.models.load_model('models/BEST.mdl_wts.hdf5')
print('AUC on test data - best model:',model4.evaluate(x_test, y_test, verbose=0)[1])

AUC on test data - last step: 0.61160696
AUC on test data - best model: 0.7024112


model3.save("models/dl-2_train82_test73_2.h5")

# Evaluating

In [32]:
# We have many trained models:  
import os
modeldir = 'models/'
models = os.listdir(modeldir)
models

['BEST.mdl_wts.hdf5',
 'custom_train78_test70.h5',
 'custom_train86_test71.h5',
 'custom_train87_test70.h5',
 'dl-1_train85_test69.h5',
 'dl-1_train88_test71.h5',
 'dl-2_train76_test68.h5',
 'dl-2_train80_test72.h5',
 'dl-2_train82_test73.h5',
 'dl-2_train82_test73_2.h5',
 'dl-2_train87_test72.h5',
 'dl-2_trainX_test71.hdf5']

In [35]:
modeldir + models[9]

'models/dl-2_train82_test73_2.h5'

In [36]:
# We have these models:
models

['BEST.mdl_wts.hdf5',
 'custom_train78_test70.h5',
 'custom_train86_test71.h5',
 'custom_train87_test70.h5',
 'dl-1_train85_test69.h5',
 'dl-1_train88_test71.h5',
 'dl-2_train76_test68.h5',
 'dl-2_train80_test72.h5',
 'dl-2_train82_test73.h5',
 'dl-2_train82_test73_2.h5',
 'dl-2_train87_test72.h5',
 'dl-2_trainX_test71.hdf5']

In [37]:
# Check the AUC of the saved models
for i in range(len(models)):
    model4 = tf.keras.models.load_model(modeldir + models[i])
    print(models[i], "- AUC: ", model4.evaluate(x_test, y_test, verbose=0)[1])

BEST.mdl_wts.hdf5 - AUC:  0.7024112
custom_train78_test70.h5 - AUC:  0.70122445
custom_train86_test71.h5 - AUC:  0.70935297
custom_train87_test70.h5 - AUC:  0.70803803
dl-1_train85_test69.h5 - AUC:  0.69341934
dl-1_train88_test71.h5 - AUC:  0.7133474
dl-2_train76_test68.h5 - AUC:  0.6785109
dl-2_train80_test72.h5 - AUC:  0.7245231
dl-2_train82_test73.h5 - AUC:  0.73281294
dl-2_train82_test73_2.h5 - AUC:  0.7352253
dl-2_train87_test72.h5 - AUC:  0.7209686
dl-2_trainX_test71.hdf5 - AUC:  0.71767056


# The best model

In [38]:
#This is the  'dl-2_train82_test73_2.h5', so let's see it.
model_best = tf.keras.models.load_model(modeldir + models[9])
model_best.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 7, 20, 256)        1536      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 7, 10, 256)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 1, 10, 128)        229504    
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 10, 128)        0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 1, 5, 128)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 640)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)             

#### This is a modified DL-2.